In [1]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

# import ind.alpha191 as a191


import matplotlib.pyplot as plt
import seaborn as sns

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates



# from QUANTAXIS.QAUtil import DATABASE
# from QUANTAXIS.QAUtil import  trade_date_sse
# from QUANTAXIS.QAUtil.QADate_trade import (
#     QA_util_get_pre_trade_date,
#     QA_util_get_next_trade_date,
#     QA_util_if_tradetime
# )

# import inspect

# from sklearn import linear_model

%load_ext autoreload
%autoreload 2
%aimport ind.alpha191

In [2]:
###加载数据

ids = np.arange(1,192)
# np.random.shuffle(ids)
files = list(map(lambda x:'191_alpha'+'0'*(3- int(np.log10(x)+1))+str(x)+'_train',ids))

def load_alpha_factors(file_names):
    from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
    from base.JuUnits import excute_for_multidates
    import numpy as np
    import pandas as pd 
    
    temp = []
    for file in file_names:
        print(file)
        temp.append(load_cache(file, cache_type=CACHE_TYPE.FACTOR))
    
    return pd.concat(temp,axis=1)
        
worker=6
task = task_chunk_split(files, worker)
results = parallal_task(worker, load_alpha_factors, task)
res = pd.concat(results,axis=1)

Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:21644  pid:25024


  0%|          | 0/6 [00:00<?, ?it/s]

In [263]:
def nan_count(x):
    return round(x.isna().sum(axis=0)/len(x),4)*100

def filter_invalid_factor(fators_df):
    ## 缺失30%以上，且3倍方差以上，废弃
    ## 加极端判断是为了保证留下那种大窗口期的因子。
    
    fators = fators_df.replace([np.inf, -np.inf], np.nan)
    
    #计算缺失率
    valid_rate = excute_for_multidates(fators, nan_count, level=1) 
    valid_distribute_boundary = valid_rate.mean(axis=0) + valid_rate.std(axis=0)*3
    #缺失率符合当前因子在所有标的上，符合3个标准差的
    filter_contition = valid_rate > valid_distribute_boundary 
    
    def filter_invalid(x):
        code = x.index.get_level_values(1)[0]
        distribute_condition = filter_contition.loc[code] 
        #符合3个标准差的列
        invalid_factors_name = distribute_condition[distribute_condition==True].index 
        #符合3个标准差，并且缺失30%以上的列
        thredhold_condition = valid_rate.loc[code][invalid_factors_name]>30 
        x[thredhold_condition.index] = np.nan 
        return x

    filtered = excute_for_multidates(fators, filter_invalid, level=1)
    
    return filtered

factors_filtered = filter_invalid_factor(res)

In [264]:
def filter_correlative_factor(fators_df, thredhold=.70, show_chart=False,verbose=False):
    #去掉相关性强的因子，相关性大于thredhold的因子（列）将被去除
    corr = fators_df.corr()
    matrix = corr.values
    #取得下三角矩阵
    matrix[np.triu_indices(matrix.shape[0])]=np.nan
    corr_triu = pd.DataFrame(matrix, columns=corr.columns, index=corr.index)

    if show_chart:
        plt.figure(figsize=(30,30))
        labels = corr.columns.str.replace('191_alpha','')
        ax = sns.heatmap(corr_triu, xticklabels=labels, yticklabels=labels, cmap="YlGnBu",cbar=True)

    # 取得符合条件的因子对后，再取“一对的左边”作为删除目标
    correlative_columns_name = corr_triu.abs().gt(thredhold).stack().loc[lambda x: x].index.get_level_values(0).unique().to_list()
    
    if verbose:
        print('delete correlative factors: {}'.format(','.join(correlative_columns_name)))
        
    correlative_droed = fators_df.drop(columns=correlative_columns_name)
    return correlative_droed

factors_filtered = filter_correlative_factor(factors_filtered,show_chart=False,verbose=True)

delete correlative factors: 191_alpha008,191_alpha014,191_alpha020,191_alpha022,191_alpha024,191_alpha026,191_alpha027,191_alpha028,191_alpha031,191_alpha034,191_alpha037,191_alpha040,191_alpha046,191_alpha047,191_alpha049,191_alpha050,191_alpha051,191_alpha052,191_alpha053,191_alpha055,191_alpha057,191_alpha058,191_alpha059,191_alpha062,191_alpha065,191_alpha066,191_alpha067,191_alpha068,191_alpha071,191_alpha072,191_alpha078,191_alpha079,191_alpha082,191_alpha088,191_alpha089,191_alpha090,191_alpha094,191_alpha095,191_alpha096,191_alpha097,191_alpha099,191_alpha100,191_alpha105,191_alpha106,191_alpha112,191_alpha116,191_alpha121,191_alpha122,191_alpha126,191_alpha128,191_alpha129,191_alpha132,191_alpha133,191_alpha134,191_alpha136,191_alpha137,191_alpha139,191_alpha143,191_alpha147,191_alpha150,191_alpha153,191_alpha154,191_alpha156,191_alpha160,191_alpha161,191_alpha162,191_alpha165,191_alpha167,191_alpha168,191_alpha173,191_alpha174,191_alpha175,191_alpha177,191_alpha180,191_alpha1

In [265]:
# with pd.option_context("display.max_rows", None, "display.max_columns", None):

factors = factors_filtered

codes = factors.index.get_level_values(1).unique()
dates = factors.index.get_level_values(0).unique()
data_replenish = smpl.get_data(codes.to_list(),dates.min().strftime('%Y-%m-%d'),dates.max().strftime('%Y-%m-%d')).data
ret_forward = smpl.get_forward_return(data_replenish,'close')
ret_forward.name = 'ret_forward'


mv_industry = smpl.add_marketvalue_industry(data_replenish).drop(columns=data_replenish.columns.to_list())
industy = mv_industry['industry']
cap = mv_industry['totalCapital']


# factors = factors.assign(cap=cap,industry = industy)
factors = factors.assign(country=1.0, cap=cap,industry = industy)
factors = pd.get_dummies(factors,'industry')

## 国家因子换到第一列
columns_ = factors.columns.to_list()
columns_.insert(0,columns_.pop(columns_.index('country')))
factors = factors.reindex(columns=columns_)



# adj_weight = np.sqrt(cap)/np.sqrt(cap).sum()
# adj_weight_mat = pd.DataFrame(np.diag(adj_weight),index=adj_weight.index, columns=adj_weight.index)

# del factors['country']



In [364]:
# factors.columns.to_list()

test = factors.loc['2021-01-05']

test_invalid_check = nan_count(test)
## 去除缺失5%以上的因子
test_invalid_checked = test.drop(columns=test_invalid_check[test_invalid_check>5].index)
## 去除仍有少量缺失的标的
test_invalid_checked = test_invalid_checked.dropna()
## 去除内容全部相同的列
test_invalid_checked = test_invalid_checked.loc[:, (test_invalid_checked != test_invalid_checked.iloc[0]).any()]
## 去除内容列后，从新计算剩余的行业数量
industy_count = test_invalid_checked.columns.str.contains('industry_').sum()


test_cap=test_invalid_checked['cap']


#计算个股权重矩阵
##个股权重矩阵，权重为流通市值平方占比
cap_temp = np.sqrt(test_cap)
wight = cap_temp/cap_temp.sum()
V = np.diag(wight) #pd.DataFrame(np.diag(wight),columns = test_cap.index,index = test_cap.index)


test_industy_dummy = test_invalid_checked.iloc[:,-1*industy_count:]
industry_cap = test_cap.T.dot(test_industy_dummy)
industry_wight = industry_cap/industry_cap.sum()




X = test_invalid_checked[test_invalid_checked.columns.difference(['cap'],sort=False)]

##计算约束矩阵防止共线性问题
##获取申万28个行业列表
            #计算行业流通市值占比
            # _industry_wgt[i] = ind_float_mv[ind_float_mv['industry']==i]['float_mv'].sum()/_sum_float_mv
        #构建约束矩阵R，R为K*K-1矩阵，K为因子数量
        #将最后一个行业因子表示为其他行业的线性形式
        #移除最后一个行业所在的列，并对其所在的行进行权重处理

k = X.shape[1] #总因子数量
p = industy_count #行业因子数量
# q = k-1-p #风格因子数量
q = k-p #原风格因子数量


diag_R = np.diag(np.ones(k))
location = q
R = np.delete(diag_R,location+p-1,axis = 1)
adj_industry_weights = -industry_wight.div(industry_wight[-1]).iloc[:-1]
R[location+p-1,location:location+p] = adj_industry_weights


# showR = pd.DataFrame(R,columns=np.delete(X.columns,location+p-1),index=X.columns)
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     display(showR)
# showR.to_excel('./showR.xlsx')

#求解投资组合股票权重矩阵
Y = ret_forward.loc['2021-01-05'][X.index]
W = R.dot(np.linalg.inv(np.array(R.T.dot(X.T).dot(V).dot(X).dot(R),dtype='float'))).dot(R.T).dot(X.T).dot(V)
W = pd.DataFrame(W,index = X.columns,columns = X.index).round(6)
#计算因子收益率
f = W.dot(Y).T
residual = Y- X.dot(f.T)
# residual.sum()


In [366]:
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     display(X.isna().sum(axis=0)>0)
# test
# test_invalid_check = nan_count(test)
# test_invalid_checked = test.drop(columns=test_invalid_check[test_invalid_check>5].index)
# test_invalid_checked.dropna()

# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     # display(pd.DataFrame(R.T.dot(X.T),columns=X.index,index=showR.columns).sum(axis=1))
#     display(pd.DataFrame(R.T.dot(X.T).dot(V).dot(X).dot(R),columns=showR.columns,index=showR.columns))
    
    # display(X.sum(axis=0))
    # display(R.T.dot(X.T))
# W

W

code,000001,000002,000063,000066,000069,000157,000166,000301,000333,000338,...,603486,603501,603659,603799,603806,603833,603882,603899,603986,603993
191_alpha001,-0.124047,0.065127,0.067990,0.165150,-0.112729,0.029211,-0.115874,-0.079044,0.323243,0.048201,...,0.035450,-0.065561,0.315616,0.076636,0.064672,0.036474,0.056724,0.049211,0.087732,-0.045046
191_alpha002,0.163788,-0.066613,-0.099875,-0.273031,0.405235,-0.056732,0.003851,0.071151,-0.520515,-0.018405,...,-0.126494,0.198605,-0.350774,-0.034801,-0.023086,-0.000265,-0.130680,-0.085429,0.052338,0.103605
191_alpha003,0.025347,-0.014178,-0.009704,-0.012086,0.030108,-0.003871,-0.060850,-0.003702,-0.034078,-0.009283,...,-0.003213,-0.023000,-0.034773,-0.012875,-0.010489,-0.006590,-0.005641,-0.007700,-0.022878,-0.013538
191_alpha004,0.185510,-0.049041,-0.096799,-0.139641,0.373462,-0.019473,-0.282600,0.031182,-0.286279,-0.027545,...,-0.075437,0.024562,-0.127622,-0.074964,-0.052196,-0.019222,-0.082317,-0.082076,-0.074346,-0.035685
191_alpha005,-1.039765,-0.407039,-0.336670,-0.019201,-0.515971,-0.082686,-0.196960,-0.335902,-0.709962,-0.202020,...,-0.271456,-0.411292,0.179922,-0.230720,-0.188672,-0.111318,-0.322059,-0.276125,-0.089727,0.016399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
industry_防水材料,0.132309,-0.643817,-0.011508,-0.490512,0.907197,-0.339696,-0.772852,0.805133,-0.651055,-0.006857,...,-0.003439,0.017938,-1.213119,-0.001325,-0.001563,-0.004298,-0.004499,-0.005666,0.339461,0.020341
industry_集成电路设计,2.036856,0.731824,-0.012034,-0.737075,0.886948,-0.081841,0.316114,0.131268,-1.275096,-0.004423,...,-0.002982,0.975489,-1.404612,-0.000689,0.000605,-0.005863,-0.002505,0.002188,0.069789,-0.015766
industry_面板,6.279739,-3.433727,0.023424,-1.722404,8.391631,2.039118,-9.415400,-5.987727,3.232670,0.007989,...,0.009000,4.716915,3.575534,0.005783,-0.011795,0.005200,0.002226,0.003871,-2.227775,0.076015
industry_风电整机,-0.778153,0.198714,-0.004316,0.029687,-0.188483,0.511068,-0.909762,0.009850,0.896163,-0.001851,...,-0.001348,0.021228,0.084348,-0.002018,-0.002049,-0.002414,-0.003003,-0.004053,0.095668,0.014640


In [370]:
f

191_alpha001       0.019567
191_alpha002      -0.090715
191_alpha003      -0.012561
191_alpha004      -0.069463
191_alpha005      -0.839697
                     ...   
industry_防水材料      0.345858
industry_集成电路设计    0.348452
industry_面板       -1.110030
industry_风电整机     -0.094338
industry_黄金        0.174766
Length: 220, dtype: float64

In [372]:
X.dot(f.T)

code
000001    0.260308
000002    0.333018
000063   -0.398317
000066   -0.140668
000069    0.145888
            ...   
603833    0.283866
603882   -0.115622
603899    0.090239
603986    0.012878
603993   -1.954388
Length: 214, dtype: float64